In [40]:
import pandas as pd
from dask import delayed, compute
import dask.dataframe as ddf

from pytz import timezone
import pandas_market_calendars as mcals
from datetime import datetime, timedelta

from urllib.parse import urlencode

import gettickers as gt
import config as cfg

TIINGO_TOKEN = cfg.TT
URL = 'https://api.tiingo.com/tiingo/daily/'
MAX_CLIENTS = 6
CSV_PATH = './CSVs'


In [2]:
TIINGO_TOKEN

'66bb0f15dd2a99842c6dd50391cf859c4c561983'

In [20]:
@delayed
def download_csv(symbol):
    mydict = {
        'token': TIINGO_TOKEN,
        'startDate': '1970-01-02',
        'resampleFreq': 'weekly',
        'format': 'csv',
        'columns': 'date,adjClose,adjHigh,adjLow,adjVolume'}
    url = URL + symbol + '/prices/?' + urlencode(mydict)
    # headers = {
    #     'Content-Type': 'text/csv'}
    # df = ddf.read_csv(url, index_col=0, parse_dates=['date'])
    df = ddf.read_csv(url, parse_dates=['date'])
    df['symbol'] = symbol
    return df

In [4]:
def csv_name(symbol):
    if datetime.now().hour > 19:
        return symbol.upper() + str(datetime.today().date()) + '.csv'
    else:
        return symbol.upper() + \
            str(datetime.today().date() -timedelta(days=1)) + '.csv'

In [5]:
@delayed
def write_csvs(args):
    # csvs = "Written to disk:\n"
    tasks = []
    for arg in args:
        symbol = arg.name
        filename = CSV_PATH + csv_name(symbol)
        task = arg.to_csv(filename, single_file=True)
        tasks.append(task)
    return tasks

In [6]:
@delayed
def join_dfs(args):
    # csvs = "Written to disk:\n"
    tasks = []
    for symbol in args:
        filename = CSV_PATH + csv_name(symbol)
        task = arg.to_csv(filename, single_file=True)
        tasks.append(task)
    return tasks

In [7]:
tickers = gt.get_tickers()

In [9]:
chunks = [tickers[i:i+MAX_CLIENTS] for i in range(0, len(tickers), MAX_CLIENTS)]

In [11]:
chunks[0:2]

[['ORC', 'PINC', 'HOFT', 'LOB', 'SEE', 'EME'],
 ['CMI', 'YORW', 'BMI', 'STAA', 'CONE', 'KMB']]

In [12]:
df = for chunk in chunks[0:2]:
        df = delayed(ddf.concat)([download_csv(i) for i in chunk], axis=0)

TypeError: Truth of Delayed objects is not supported

In [23]:
l = [download_csv(i).compute() for i in chunks[0]]

In [15]:
df = ddf.concat(l.compute(), axis=0)

AttributeError: 'list' object has no attribute 'compute'

In [21]:
wfc = download_csv('WFC')

In [22]:
wfc.head().compute()

,date,adjClose,adjHigh,adjLow,adjVolume,symbol
0,1972-06-02,0.334109,0.337105,0.333163,422400,WFC
1,1972-06-09,0.337105,0.344991,0.333163,734400,WFC
2,1972-06-16,0.341048,0.341048,0.335134,600000,WFC
3,1972-06-23,0.345937,0.345937,0.337105,336000,WFC
4,1972-06-30,0.362733,0.362733,0.343020,374400,WFC


In [24]:
jpm = download_csv('JPM')

In [25]:
df = ddf.concat([wfc, jpm])

TypeError: Truth of Delayed objects is not supported

In [39]:
pd.Timestamp(datetime.now(tz=timezone('America/New_York')))

Timestamp('2020-05-10 17:52:35.795400-0400', tz='America/New_York')

In [41]:
nyse = mcals.get_calendar('NYSE')

In [53]:
sched = nyse.schedule(start_date=str(datetime.today() - timedelta(days=3)), end_date=str(datetime.today()))

In [56]:
nyse.open_at_time(sched, pd.Timestamp(datetime.now(tz=timezone('America/New_York')) - timedelta(days=2, hours=2)))

False